The purpose of this notebook is to quantify the interannual variability in the tropical humidity distribution in the ERA5 data

In [ ]:
import numpy as np
import os
import random
import pickle
import seaborn
import logging
import typhon
import filenames
from pylab import cm
from os.path import join
from scipy.interpolate import interp1d
from scipy.signal import detrend
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import postprocessing_tools as ptools
from importlib import reload
from moisture_space import utils, plots, moisture_space
mpl_logger = logging.getLogger('matplotlib') 
mpl_logger.setLevel(logging.WARNING)
plt.style.use(typhon.plots.styles.get('typhon'))   

# Config

In [ ]:
years = [2014, 2015, 2016, 2017, 2018]
time_period = {}
for y in years:
    time_period[y] = [f'{y}-08-10', f'{y}-09-08']

    
model = 'ERA5'
run = '31.0km'
exp = 'ERA5-31.0km'
variables_3D = ['TEMP', 'PRES', 'QV', 'QI', 'QC', 'RH', 'W']
variables_2D = []
variables = variables_3D + variables_2D
data_dir = '/mnt/lustre02/work/mh1126/m300773/DYAMOND'
filename_suffix = ''
num_samples = 10_000_000
num_percs = 50
h = np.arange(100, 17900, 400) # height vector to interpolate on       

# Read data

In [ ]:
ms = {}
for y in years:
    ms[y] = {}
    for var in variables:
        filename = filenames.averaged_profiles(data_dir, model, run, var, num_samples, num_percs, time_period[y], filename_suffix)
        ms[y][var] = moisture_space.PercMoistureSpace.from_netcdf(filename)
        if var in variables_3D:
            ms[y][var] = ms[y][var].interpolate(h, kind='linear', interp_space='linear')
perc_values = ms[y][var].bins

# Create collection of moisture spaces from different years

In [ ]:
reload(moisture_space)
ms_series = {}
for var in variables_3D:
    perc_space_list = [ms[year][var] for year in years]
    ms_series[var] = moisture_space.MoistureSpaceSeries(perc_space_list)

# Mean profiles

In [ ]:
# Save mean profiles to .pkl file
mean_profiles = {}
mean_profiles['height'] = h
for n, year in enumerate(years):
    mean_profiles[year] = {}
    for var in variables_3D:
        mean_profiles[year][var] = ms[year][var].mean_profile()

# with open(os.path.join(datapath, f'ERA5_tropical_mean_profiles_{years[0]}-{years[-1]}.pkl'), 'wb') as outfile:
#     pickle.dump(mean_profiles, outfile)
        

In [ ]:
reload(plots)
plt.rcParams.update({'font.size': 15})
plot_vars = ['TEMP', 'QV', 'RH', 'QI', 'QC']
num_vars = len(plot_vars)
bin_start= 0
bin_end = 100
plt.rcParams.update({'font.size': 17})
fig, ax = plt.subplots(1, num_vars, figsize=(5 * num_vars, 7), sharey=True)
for i, var in enumerate(plot_vars):
    for year in years:
        if var == 'QV':
            ax[i].semilogx(ms[year][var].mean_profile(bin_start, bin_end) * plots.scaling_factors(var),
                           h * 1e-3, lw=2.5, label=year)
        else:
            ax[i].plot(ms[year][var].mean_profile(bin_start, bin_end) * plots.scaling_factors(var),\
                       h * 1e-3, lw=2.5, label=year)
        ax[i].set_xlabel(plots.axis_labels(var))
        
        ax[i].set_ylim(0, 17.5)
        seaborn.despine(ax=ax[i])
ax[0].set_ylabel('Height [m]')
#ax[1].set_xlim(0, 20)
ax[0].set_xlim(180, 300)
ax[2].set_xlim(0, 90)
ax[3].set_xlim(0, 15)
ax[4].set_xlim(0, 50)
ax[0].legend(fontsize=12)
plt.tight_layout()

# Distribution of RH in moisture space

In [ ]:
reload(plots)
fig, ax = plt.subplots(np.ceil(len(years) / 2).astype(int), 2, figsize=(15, 15))
tick_labels = np.arange(10, 81, 10)
ax = ax.ravel()

plt.rcParams.update({'font.size': 20})
x_lims = [0, 100]
y_lims = [0, 17.5]
x_label = 'Percentile of IWV'#axis_labels['IWV']
y_label = 'Height [km]'
contours = np.arange(0, 101, 5)
contours_ci = np.arange(1, 200, 20)
contours_cl = contours_ci

lower_layer_bnd = 100
upper_layer_bnd = 12500

for i, year in enumerate(years):
    plots.moisture_space_contourf(fig, ax[i], perc_values, h * 1e-3, (ms[year]['RH'].mean.T) * 1e2, contours,\
                                  x_lims, y_lims, x_label, y_label, 'RH [%]',
                                  cm_orientation='vertical', cb_extend='neither', cb_ticks=np.arange(0, 110, 10),\
                                  cmap='BrBG', vmin=0, vmax=100)
        
    ci = plots.moisture_space_contour(fig, ax[i], perc_values, h * 1e-3, ms[year]['QI'].mean.T * 1e6, contours_ci,\
                                  x_lims, y_lims, x_label, y_label, 'w')
    cl = plots.moisture_space_contour(fig, ax[i], perc_values, h * 1e-3, ms[year]['QC'].mean.T * 1e6, contours_cl,\
                                   x_lims, y_lims, x_label, y_label, 'lightblue')
    cs = plots.moisture_space_contour(fig, ax[i], perc_values, h * 1e-3, ms[year]['W'].mean.T,\
                             [0.0], x_lims, y_lims, x_label, y_label, 'k')
    #cs = plots.moisture_space_contour(fig, ax[i], perc_values, h * 1e-3, ms[exp]['W'].mean.T,\
    #                         [0.0], x_lims, y_lims, x_label, y_label, 'k')
#     plots.moisture_space_line(ax[i], perc_values, ms[year]['H_tropo'].mean * 1e-3,\
#                                   x_lims, y_lims, x_label, y_label, lw=2, label=year, color='r')
    ax[i].set_title(year)

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
x_lims = [0, 100]
y_lims = [0, 17.5]
x_label = 'Percentile of IWV'#axis_labels['IWV']
y_label = 'Height / km'
plots.moisture_space_contourf(fig, ax, perc_values, h * 1e-3, ms_series['RH'].variability.T * 1e2, np.arange(0, 13, 1),\
                              x_lims, y_lims, 'Percentile of IWV', y_label, 'RH standard deviation / %',\
                              cmap='Reds', cm_orientation='vertical', cb_extend='neither', vmin=0, vmax=12)
plt.savefig('plots_seminar/ERA5_interannual_variability.pdf', dpi=400)

fig, ax = plt.subplots(figsize=(10, 5))
x_lims = [0, 100]
y_lims = [0, 17.5]
x_label = 'Percentile of IWV'#axis_labels['IWV']
y_label = 'Height / km'
plots.moisture_space_contourf(fig, ax, perc_values, h * 1e-3, ms_series['RH'].variability.T / ms_series['RH'].mean.T * 1e2, np.arange(0, 31, 2.5),\
                              x_lims, y_lims, 'Percentile of IWV', y_label, 'RH standard deviation / %',\
                              cmap='Reds', cm_orientation='vertical', cb_extend='neither', vmin=0, vmax=30)
plt.savefig('plots_seminar/ERA5_interannual_variability.pdf', dpi=400)

# Distribution of w in moisture space

In [ ]:
reload(plots)
fig, ax = plt.subplots(np.ceil(len(years) / 2).astype(int), 2, figsize=(15, 10))
tick_labels = np.arange(10, 81, 10)
ax = ax.ravel()

plt.rcParams.update({'font.size': 20})
x_lims = [0, 100]
y_lims = [0, 17.5]
x_label = 'Percentile of IWV'#axis_labels['IWV']
y_label = 'Height [km]'
contours = np.arange(-0.02, 0.020001, 0.0005)

lower_layer_bnd = 100
upper_layer_bnd = 12500

for i, year in enumerate(years):
    plots.moisture_space_contourf(fig, ax[i], perc_values, h * 1e-3, (ms[year]['W'].mean.T), contours,\
                                  x_lims, y_lims, x_label, y_label, 'W [m s-1]',
                                  cm_orientation='vertical', cb_extend='both',\
                                  cmap='difference')
    ax[i].set_title(year)